In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from datetime import datetime
import sys
import math
import seaborn as sns
import matplotlib.pyplot as plt
if 'holidays' not in sys.modules:
    !pip install holidays
import holidays

In [2]:
driver_df = pd.read_csv('driver_ids.csv', parse_dates = ['driver_onboard_date'])
ride_df = pd.read_csv('ride_ids.csv')
ride_timestamps_df = pd.read_csv('ride_timestamps.csv', parse_dates = ['timestamp'])

'''
rides info combined dataframe
'''
big_rides_info = pd.read_csv('added_variables_rides_info.csv', parse_dates = ['driver_onboard_date', 'requested_at','accepted_at',
                                                                             'arrived_at','picked_up_at','dropped_off_at'])
print(big_rides_info.shape)
display(big_rides_info.head())

'''
drivers info combined dataframe
'''
big_drivers_info = pd.read_csv('temp_added_drivers_info.csv', parse_dates = ['driver_onboard_date', 'last_ride']).set_index('driver_id')
print(big_drivers_info.shape)
display(big_drivers_info.head())

'''
driving intervals
'''
percentile = pd.read_csv('driving_interval.csv').set_index('driver_id')
print(percentile.shape)
display(percentile.head())

(184209, 18)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at,speed(m/s),holiday,weekend,time of day,season,time spent to arrive at the customer(minutes),fare
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29,2016-04-23 02:13:50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:16:40,2016-04-23 02:22:07,5.538226,0,1,4,0,2.766667,9.360365
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29,2016-03-29 19:00:49,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:04:01,2016-03-29 19:17:30,4.155748,0,0,3,0,3.133333,9.113439
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29,2016-06-21 11:56:31,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:01:35,2016-06-21 12:11:07,5.737762,0,0,1,1,5.016667,8.187399
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29,2016-05-19 09:15:29,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 09:18:20,2016-05-19 10:13:58,19.557519,0,0,1,0,2.850000,78.170140
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29,2016-04-20 22:05:30,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:07:02,2016-04-20 22:20:45,5.000000,0,0,3,0,1.550000,19.403323


(837, 22)


,prime time rides percentage,average daily active time(hrs/day),gross fare(over all rides),average daily fare,average fare per ride,average monthly fare,total rides,number of abnormal rides,completion rate,active days,...,driver_onboard_date,total days with Lyft,Retention Rate,Lifetime Value,Lifetime,Work_Period,Value,Average Speed,Average Single Ride Duration,Average Single Ride Distance
driver_id,,,,,,,,,,,,,,,,,,,,,
002be0ffdc997bd5c50703158b7c2491,0.397112,1.097411,3651.957780,65.213532,13.183963,912.989445,277,0,1.0,56,...,2016-03-29,90,0.622222,4756.038040,2.093023,Day,Medium,6.690127,798.693141,6282.624549
007f0389f9c7b03ef97098422f902e62,0.387097,0.474468,332.355469,27.696289,10.721144,83.088867,31,0,1.0,12,...,2016-03-29,90,0.133333,45.842134,1.034483,Day,Low,5.706670,661.193548,3791.322581
011e5c5dfc5c2c92501b8b24d47509bc,0.470588,0.676042,493.765123,41.147094,14.522504,164.588374,34,0,1.0,12,...,2016-04-05,83,0.144578,74.718556,1.046658,Day,Low,7.013431,858.970588,7930.970588
0152a2f305e71d26cc964f8d4411add9,0.251309,1.211951,2644.233047,66.105826,13.844152,881.411016,191,0,1.0,40,...,2016-04-23,65,0.615385,3357.756250,2.063492,Night,Medium,7.735094,913.722513,7702.821990
01674381af7edd264113d4e6ed55ecda,0.264000,2.482243,5457.997548,136.449939,14.554660,1819.332516,375,0,1.0,40,...,2016-04-29,59,0.677966,8767.867546,2.369478,Day,High,7.498338,953.181333,8329.717333


(837, 20)


,mean,std,min,2.5%,5%,10%,12.5%,25%,50%,75%,87.5%,90%,95%,97.5%,max,duration mid 95%,duration mid 90%,duration mid 80%,duration mid 75%,duration mid 50%
driver_id,,,,,,,,,,,,,,,,,,,,
7ff85c5c0e9324e28d1e0d0589c364bd,15.033426,0.323981,14.740000,14.751958,14.763917,14.787833,14.799792,14.859583,14.979167,15.180139,15.280625,15.300722,15.340917,15.361014,15.381111,0.609056,0.577000,0.512889,0.480833,0.320556
7b625f643d0775f0ac4898e33235377b,18.056574,1.258248,16.604722,16.711222,16.817722,17.030722,17.137222,17.669722,18.734722,18.782500,18.806389,18.811167,18.820722,18.825500,18.830278,2.114278,2.003000,1.780444,1.669167,1.112778
0822973f3e812a68a867962dbbd37f5d,23.756682,0.795498,22.552500,22.623472,22.652153,22.786667,22.879965,23.193611,23.771111,24.248194,24.669549,24.736389,24.840000,25.024167,26.208333,2.400694,2.187847,1.949722,1.789583,1.054583
80cb80668a3c444e3e8962d1c861b9d0,21.527642,0.669526,19.926944,20.026986,20.195333,20.661833,20.766875,21.226528,21.510556,21.869722,22.195347,22.356667,22.467833,22.657903,23.109722,2.630917,2.272500,1.694833,1.428472,0.643194
1805f90c5220ac830835df3309a76e78,18.330698,1.364033,12.123611,16.851111,17.470611,17.509500,17.590417,17.946111,18.290556,18.683056,19.616111,20.022333,20.362944,20.635333,20.760833,3.784222,2.892333,2.512833,2.025694,0.736944
